<a href="https://colab.research.google.com/github/macorony/Workshop-of-Parallel-Programming-in-Julia/blob/main/Multi_threading_with_Base_Threads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [1]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.7.1" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [2]:
versioninfo()

Julia Version 1.7.1
Commit ac5cc99908 (2021-12-22 19:35 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 2


In [3]:
using BenchmarkTools

M = rand(2^11, 2^11)

@btime $M * $M;

  460.962 ms (2 allocations: 32.00 MiB)


In [4]:
if ENV["COLAB_GPU"] == "1"
    using CUDA

    run(`nvidia-smi`)

    # Create a new random matrix directly on the GPU:
    M_on_gpu = CUDA.CURAND.rand(2^11, 2^11)
    @btime $M_on_gpu * $M_on_gpu; nothing
else
    println("No GPU found.")
end

No GPU found.


# Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.

Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />

In [5]:
using Base.Threads
nthreads()

2

In [6]:
@threads for i=1:10
println("iteration $i on thread $(threadid())")
end

iteration 1 on thread 1
iteration 2 on thread 1
iteration 3 on thread 1
iteration 4 on thread 1
iteration 5 on thread 1
iteration 6 on thread 2
iteration 7 on thread 2
iteration 8 on thread 2
iteration 9 on thread 2
iteration 10 on thread 2


This will split the loop task to 2 threads on two cores.

In [7]:
a = zeros(10)
@threads for i = 1:10
      a[i] = threadid()
end

In [8]:
a

10-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 2.0
 2.0
 2.0
 2.0
 2.0

This array is filled in parallel and separate. This is called thread-safe.

In [9]:
nthreads()
n = Int64(1e8)
a = zeros(n)
typeof(a)

Vector{Float64} (alias for Array{Float64, 1})

In [10]:
# single thread
@time for i in 1:n
a[i] = log10(i)
end

 18.957808 seconds (400.05 M allocations: 7.453 GiB, 8.55% gc time, 0.13% compilation time)


In [11]:
# 2 threads
using Base.Threads
@time @threads for i in 1:n
a[i] = log10(i)
end

  6.521849 seconds (200.02 M allocations: 2.981 GiB, 17.59% gc time, 0.54% compilation time)


In [12]:
total = 0
@threads for i = 1:Int(1e6)
global total += i
end
println("total = ", total)

total = 262699587711


It is not thread-safe code
1. race condition
2. result varies every time

The problem can be solved by an atomic variable total. The idea is that only one thread can update atomic variable at a time. All other threads wait for the variable to be released.

In [13]:
total = Atomic{Int64}(0)
@threads for i in 1:Int(1e6)
atomic_add!(total, i)
end
print("total = ", total[])

total = 500000500000

Although we get the same result. But the code is much slower because threads must wait the variable to be released.

In [14]:
n = Int64(1e8)
total = Int128(0)
@time for i in 1:n
global total += i
end
println("total = ", total)

 21.539827 seconds (400.00 M allocations: 8.941 GiB, 8.88% gc time, 0.03% compilation time)
total = 5000000050000000


In [15]:
using BenchmarkTools
n = Int64(1e8)
total = Int128(0)
@btime for i in 1:n
global total += i
end
print("total = ", total)

  16.217 s (399998979 allocations: 8.94 GiB)
total = 20000000200000000

In [16]:
function quick(n)
total = Int128(0)
  for i in 1:n
    total += i
  end
  return(total)
end

quick (generic function with 1 method)

In [17]:
@btime quick(Int64(1e8))
@btime quick(Int64(1e9))
@btime quick(Int64(1e15))

  2.252 ns (0 allocations: 0 bytes)
  2.252 ns (0 allocations: 0 bytes)
  1.886 ns (0 allocations: 0 bytes)


500000000000000500000000000000

# Slow series converge

In [18]:
# An integer exclusion function checks whether some digits in the integer numbers
function digitsin(digits::Int, num)
  base = 10
  while (digits ÷ base > 0)
    base *=10
  end
  while num > 0
    if (num % base) == digits
      return true
    end
    num ÷=10
  end
  return false
end


digitsin (generic function with 1 method)

In [23]:
function slow(n::Int64, digits::Int)
  total = Float64(0)
  for i in 1:n
    if !digitsin(digits, i)
      total += 1.0 / i
    end
  end
  return total
end


slow (generic function with 1 method)

In [24]:
@btime slow(Int64(1e8), 9)

  2.907 s (0 allocations: 0 bytes)


13.277605949858103

# Multi-thread using atomic variable

In [28]:
# Use atomic variable to rewrite the code. despite multiple threading, there will be a lot of waiting time. 
using Base.Threads
using BenchmarkTools
function slow(n::Int64, digits::Int)
  total = Atomic{Float64}(0)
  @threads for i in 1:n
    if !digitsin(digits, i)
      atomic_add!(total, 1.0 / i)
    end
  end
  return total[]
end

slow (generic function with 1 method)

In [26]:
@btime slow(Int64(1e8), 9)

  2.740 s (12 allocations: 1.03 KiB)


13.277605949858101

# Multi-thread updating separately
Each thread is updating its own sum while reducing waiting time.

In [32]:
using Base.Threads
using BenchmarkTools
function slow(n::Int64, digits::Int)
  total = zeros(Float64, nthreads())
  @threads for i in 1:n
    if !digitsin(digits, i)
      total[threadid()] += 1.0 / i
    end
  end
  return sum(total)
end

slow (generic function with 1 method)

In [33]:
@btime slow(Int64(1e8), 9)

  2.557 s (13 allocations: 1.12 KiB)


13.277605949855722

False sharing effect arises when several threads are writing into variables placed close enough to each other to end up in the same cache line. Cache lines are chunks of memory handled by the cache. If any two threads are updating variable that end up in the same cache line, the cache line will have to migrate between the two threads' caches at the cost of performance.
In general, we want to align shared global data to cache line boundaries, or avoid storing thread-specific data in an array indexed by the thread id or rank. The solution is to introduce some spacing between array elements by using function *space()* so that data from different threads do not end up in the same cache line

# Optimized function

In [34]:
function space(n::Int64, digits::Int)
  space = 8
  total = zeros(Float64, nthreads()*space)
  @threads for i in 1:n
    if !digitsin(digits, i)
      total[threadid()*space] += 1.0 / i
    end
  end
  return sum(total)
end

space (generic function with 1 method)

In [36]:
@btime space(Int64(1e8), 9)

  2.698 s (13 allocations: 1.23 KiB)


13.277605949855722

In [37]:
@btime slow(Int64(1e8), 9)

  2.555 s (13 allocations: 1.12 KiB)


13.277605949855722

# Multi-thread using heavy loops
Classical task parallelism. We divide the sum into pieces, each to be processed by an individual thread

In [43]:
using Base.Threads
using BenchmarkTools
function slow(n::Int64, digits::Int)
  numthreads = nthreads()
  threadSize = floor(Int64, n/numthreads)
  total = zeros(Float64, numthreads)
  @threads for threadid in 1:numthreads
    local start = (threadid-1)*threadSize + 1
    local finish = threadid < numthreads ? (threadid-1)*threadSize + threadSize : n
    println("thread $threadid: from $start to $finish")
    for i in start:finish
      if !digitsin(digits, i)
        total[threadid] += 1.0 / i
      end
    end
  end
  return sum(total)
end

slow (generic function with 1 method)

In [44]:
@btime slow(Int64(1e8), 9)

thread 1: from 1 to 50000000
thread 2: from 50000001 to 100000000
thread 1: from 1 to 50000000
thread 2: from 50000001 to 100000000
thread 1: from 1 to 50000000
thread 2: from 50000001 to 100000000
thread 1: from 1 to 50000000
thread 2: from 50000001 to 100000000
thread 1: from 1 to 50000000
thread 2: from 50000001 to 100000000
thread 1: from 1 to 50000000
thread 2: from 50000001 to 100000000
thread 1: from 1 to 50000000
thread 2: from 50000001 to 100000000
  2.690 s (84 allocations: 3.23 KiB)


13.277605949855722